<a href="https://colab.research.google.com/github/lvscious/Com-Aided-Case-Study/blob/main/Final_Arima_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Installing necessary Libraries
install.packages("forecast")
install.packages("tseries")
install.packages("ggplot2")

In [ ]:
# Load libraries
library(forecast)
library(tseries)
library(ggplot2)

# 2. Loading dataset
data <- read.csv("/content/full_cleaned_dataset (1).csv")
data$GDP <- as.numeric(gsub(",", "", data$GDP))
data <- na.omit(data)

# Add Year and Quarter columns to the dataframe based on assumed start and frequency
num_quarters_in_data <- nrow(data)
start_year_ts <- 2000
start_quarter_ts <- 1

# Generate a sequence of years and quarters corresponding to the data rows
all_years <- rep(start_year_ts:(start_year_ts + ceiling(num_quarters_in_data / 4) - 1), each = 4)[1:num_quarters_in_data]
all_quarters <- rep(1:4, ceiling(num_quarters_in_data / 4))[1:num_quarters_in_data]

data$Year <- all_years
data$Quarter <- all_quarters

# Filter the data to end at 2025 Q4
filtered_data <- subset(data, Year < 2025 | (Year == 2025 & Quarter <= 4))

# Create the time series object from the filtered data
gdp_ts <- ts(filtered_data$GDP, start = c(2000, 1), frequency = 4)

# 3. Stationary check (ADF Test)
log_gdp_ts <- log(gdp_ts)
adf_test <- adf.test(log_gdp_ts)
cat("ADF p-value for log GDP:", adf_test$p.value, "\n")

# 4. Differencing to Achieve Stationarity
diff_log_gdp_ts <- diff(log_gdp_ts)  # First difference
adf_test_diff <- adf.test(diff_log_gdp_ts)
cat("ADF p-value after first differencing:", adf_test_diff$p.value, "\n")

if (adf_test_diff$p.value > 0.05) {
  diff_log_gdp_ts <- diff(diff_log_gdp_ts, lag = 4)  # Seasonal difference
  adf_test_seasonal <- adf.test(diff_log_gdp_ts)
  cat("ADF p-value after seasonal differencing:", adf_test_seasonal$p.value, "\n")
}

# 5. ACF and PACF Plotting (Model Order Identification)
acf(diff_log_gdp_ts, main = "ACF of Differenced Log GDP")
pacf(diff_log_gdp_ts, main = "PACF of Differenced Log GDP")

# 6. Arima Model Selection
auto_model <- auto.arima(diff_log_gdp_ts, seasonal = FALSE)
summary(auto_model)

# 7. Model Fitting and Forecasting
# Train-Test Split
train_prop <- 0.8
n <- length(diff_log_gdp_ts)
train_size <- floor(train_prop * n)
train_ts <- window(diff_log_gdp_ts, end = c(start(diff_log_gdp_ts)[1] + (train_size - 1) %/% 4, (train_size - 1) %% 4 + 1))
test_ts <- window(diff_log_gdp_ts, start = c(start(diff_log_gdp_ts)[1] + train_size %/% 4, train_size %% 4 + 1))

# Fit the model on training data
fitted_model <- Arima(train_ts, order = arimaorder(auto_model), seasonal = list(order = c(0,0,0)))  # Ensure no seasonal

# Forecast on test set
forecast_test <- forecast(fitted_model, h = length(test_ts))

# Final forecast on full data
full_model <- auto.arima(diff_log_gdp_ts, seasonal = FALSE)  # Refit on full data, non-seasonal
future_forecast <- forecast(full_model, h = 8)

# Back-transform future forecasts to original scale
last_log_gdp_full <- log_gdp_ts[length(log_gdp_ts)]
last_year_gdp <- end(gdp_ts)[1]
last_quarter_gdp <- end(gdp_ts)[2]

if (last_quarter_gdp == frequency(gdp_ts)) {
  forecast_start_year <- last_year_gdp + 1
  forecast_start_quarter <- 1
} else {
  forecast_start_year <- last_year_gdp
  forecast_start_quarter <- last_quarter_gdp + 1
}
future_start_c <- c(forecast_start_year, forecast_start_quarter)

future_mean_log_levels <- cumsum(c(last_log_gdp_full, future_forecast$mean))[-1]
future_lower_log_levels <- cumsum(c(last_log_gdp_full, future_forecast$lower[, 2]))[-1]
future_upper_log_levels <- cumsum(c(last_log_gdp_full, future_forecast$upper[, 2]))[-1]

future_original <- ts(exp(future_mean_log_levels), start = future_start_c, frequency = frequency(gdp_ts))
future_lower <- ts(exp(future_lower_log_levels), start = future_start_c, frequency = frequency(gdp_ts))
future_upper <- ts(exp(future_upper_log_levels), start = future_start_c, frequency = frequency(gdp_ts))

# Create forecast table
forecast_table <- data.frame(
  Quarter = paste0("Q", cycle(future_original)),
  Year = floor(as.numeric(time(future_original))),
  `Point Forecast (Billions PHP)` = formatC(as.numeric(future_original), format = "f", big.mark = ",", digits = 2),
  `Lower 95% Bound` = formatC(as.numeric(future_lower), format = "f", big.mark = ",", digits = 2),
  `Upper 95% Bound` = formatC(as.numeric(future_upper), format = "f", big.mark = ",", digits = 2)
)
print(forecast_table)

# 8. Model Evaluation
checkresiduals(fitted_model)
lb_test <- Box.test(residuals(fitted_model), lag = 10, type = "Ljung-Box")
cat("Ljung-Box p-value:", lb_test$p.value, "\n")

# Back-transform test forecasts to original scale for metrics
last_log_gdp_before_test <- log_gdp_ts[train_size]
test_actual_log_levels <- cumsum(c(last_log_gdp_before_test, test_ts))[-1]
forecast_log_levels <- cumsum(c(last_log_gdp_before_test, forecast_test$mean))[-1]

test_actual_original <- ts(exp(test_actual_log_levels), start = start(test_ts), frequency = frequency(log_gdp_ts))
test_forecast_original <- ts(exp(forecast_log_levels), start = start(test_ts), frequency = frequency(log_gdp_ts))

calc_metrics <- function(actual, predicted) {
  rmse <- sqrt(mean((actual - predicted)^2, na.rm = TRUE))
  mae <- mean(abs((actual - predicted)), na.rm = TRUE)
  mape <- mean(abs((actual - predicted) / actual) * 100, na.rm = TRUE)
  return(c(RMSE = rmse, MAE = mae, MAPE = mape))
}

metrics <- calc_metrics(test_actual_original, test_forecast_original)
cat("Model Metrics:", metrics, "\n")
print(metrics)

# Plot test set forecast
autoplot(test_ts) +
  autolayer(ts(forecast_test$mean, start = start(test_ts), frequency = frequency(test_ts)), series = "Forecast") +
  ggtitle("Test Set Forecast (Differenced Scale)") +
  theme_minimal()

# Plot full series with forecast
full_log_gdp_ts_reconstructed_values <- cumsum(c(log(gdp_ts[1]), diff_log_gdp_ts))
full_ts_original <- ts(exp(full_log_gdp_ts_reconstructed_values), start = start(gdp_ts), frequency = frequency(gdp_ts))

fitted_diff <- fitted(full_model)
fitted_log_levels_values <- cumsum(c(log(gdp_ts[1]), fitted_diff))
fitted_original <- ts(exp(fitted_log_levels_values), start = start(gdp_ts), frequency = frequency(gdp_ts))

autoplot(full_ts_original) +
  autolayer(fitted_original, series = "Fitted") +
  autolayer(future_original, series = "Forecast") +
  autolayer(future_lower, series = "Lower 95%", linetype = "dashed") +
  autolayer(future_upper, series = "Upper 95%", linetype = "dashed") +
  ggtitle("GDP Forecast (Original Scale)") +
  xlab("Year") +
  ylab("GDP (Billions PHP)") +
  theme_minimal()